In [27]:
#2020/2/27 杨树澄
#整合测试文件

In [28]:
import time
from SFQ_lib import *
from Astar_lib import *
from Layout_lib import *
from Netlist_lib import *

In [29]:
#读网表信息
File_dir="whatever.v"#这里的网表路径，要从skill中获得
netlist_test=read_netlist(File_dir)
print(netlist_test)

[['auto_route'], [[[1, 'AI1', 'AI2', 'BI', 'TI1', 'TI2']]], [[[1, 'ABO']]], [[<SFQ_lib.and_e object at 0x000002771AD89828>, <SFQ_lib.cb_a object at 0x0000027718FEE208>, <SFQ_lib.d22_a object at 0x000002771AE887F0>]]]


In [30]:
#网表第一个module转dictionary
dict_info=inmod_inst_to_wire(netlist_test[3][0])
print(dict_info)

{'wireABO_net14': 'I1', 'wireAI_AI2': 'I1', 'wireBI_BI': 'I1', 'wireTI_TI2': 'I1', 'wireABO_ABO': 'I3', 'wireAI_net13': 'I3', 'wireBI_net14': 'I3', 'wireAI_AI1': 'I2', 'wireTI_TI1': 'I2', 'wireTO_net13': 'I2'}


In [31]:
#查找互联
connection=read_connection(netlist_test,dict_info)
print(connection)

['I3,wireAI,I2,wireTO', 'I3,wireBI,I1,wireABO']


In [32]:
#处理互联信息
connection_info=connect_info_process(connection)
print(connection_info)

[['I3', 'wireAI', 'I2', 'wireTO'], ['I3', 'wireBI', 'I1', 'wireABO']]


In [33]:
#读layout信息
file_list=["routing_name.txt","routing_orient.txt","routing_bbox.txt", "routing_xy.txt","routing_inst.txt"]
layout_info=layout_info_summary(file_list)
print(layout_info)

[<SFQ_lib.and_e object at 0x000002771AE885C0>, <SFQ_lib.d22_a object at 0x000002771AEA1860>, <SFQ_lib.cb_a object at 0x000002771AEA1898>]


In [34]:
#读layout信息写入list
list_layout_info=read_layout(file_list)
print(list_layout_info)

[['and_e_3x3_ai1ti2bi3abo8', 'd22_a_2x2_ai1ti3to5', 'cb_a_2x2_ai1bi2abo5'], ['MYR90', 'R0', 'MX'], [[-180, -90, -90, 0], [-60, 120, 0, 180], [270, 90, 330, 150]], [[-90, 0], [-60, 120], [270, 150]], ['I1', 'I2', 'I3']]


In [35]:
#读layout信息写入dictionary
dict_layout=layout_to_dict(layout_info)
print(dict_layout)

{'I1': {'wireAI': 1, 'area': [3, 3], 'orient': 'MYR90', 'origin': [-90, 0], 'wireTI': 2, 'wireBI': 3, 'wireABO': 8}, 'I2': {'wireAI': 1, 'area': [2, 2], 'orient': 'R0', 'origin': [-60, 120], 'wireTI': 3, 'wireTO': 5}, 'I3': {'wireAI': 1, 'area': [2, 2], 'orient': 'MX', 'origin': [270, 150], 'wireBI': 2, 'wireABO': 5}}


In [36]:
#获取connection信息对应的，两个inst的坐标和index
coord_info=get_route_coord(connection_info,dict_layout)
print(coord_info)

[[[[270, 105], 1], [[0, 135], 3]], [[[270, 135], 1], [[-135, -90], 2]]]


In [37]:
#获取版图的最大上下值 来自list_layout_info

x_0=[x[0] for x in list_layout_info[2]]
x_2=[x[2] for x in list_layout_info[2]]
y_1=[y[1] for y in list_layout_info[2]]
y_3=[y[3] for y in list_layout_info[2]]
x_max=max([max(x_0),max(x_2)])
y_max=max([max(y_1),max(y_3)])
x_min=min([min(x_0),min(x_2)])
y_min=min([min(y_1),min(y_3)])
print(x_max,x_min)
print(y_max,y_min)

330 -180
180 -90


In [38]:
#设置寻路的地图大小
width=int((x_max-x_min)/30+4)
height=int((y_max-y_min)/30+4)
origin=[x_min,y_min]
print(width,height)
print(origin)
map_origin=[int(x_min/30-2),int(y_min/30-2)]#获得一个原点为00的，map相对于layout的坐标（这里2和上面的4以后要调整）
print(map_origin)

21 13
[-180, -90]
[-8, -5]


In [40]:
#创立map
layout_map=Map(width,height)
#layout_map.showMap()
len_layout=len(layout_info)
for i in range(0,len_layout):
    #print(layout_info[i].area)
    block_point=origin_to_blockpoint(layout_info[i].area,layout_info[i].xy,layout_info[i].orient)
    #print(block_point)
    abs_point=get_abs_block_point(map_origin,block_point)
    #print(abs_point)
    layout_map.setBlock(abs_point)
    #layout_map.showMap()
#layout_map.showMap()
#开始寻路
#print(coord_info)#提示一下要连线的路径
coord_len=len(coord_info)
text=open("./createinst.il",'w+')
cellid='''cellID=dbOpenCellViewByType("ysc_layout" "auto_route" "layout" "maskLayout")'''#这里以后改，dbopen函数
print(cellid,file=text)
for i in range(0,coord_len):#????coord怎么和layout def里面的不一样
    #print(coord_info[i])
    port_1_temp=port_coord_to_map(coord_info[i][0][0],coord_info[i][0][1])#获取第一个端口坐标 map中的
    print(port_1_temp)
    port_1=[port_1_temp[0]-map_origin[0],port_1_temp[1]-map_origin[1]]#获取第一个端口的绝对坐标 map中的
    print(port_1)
    port_2_temp=port_coord_to_map(coord_info[i][1][0],coord_info[i][1][1])#同上
    print(port_2_temp)
    port_2=[port_2_temp[0]-map_origin[0],port_2_temp[1]-map_origin[1]]#同上上
    print(port_2)
    source=(port_2[0],port_2[1])#获取出发点
    dest=(port_1[0],port_1[1])#获取终点
    path=AStarSearch(layout_map,source,dest)#A星算path
    print(path)
    layout_map.showMap()
    layout_map.setBlock(path)#把这条path添加到库中
    temp=[coord_info[i][1],coord_info[i][0]]#只能用temp来重新排一下coord_info了
    #print(temp)
    new_path=[]
    len_path=len(path)
    for k in range(0,len_path):
        new_path.append([path[k][0]+map_origin[0],path[k][1]+map_origin[1]])#把还原的path坐标写入new path中
    print(new_path)
    script=path_to_inst(new_path,temp,0,connection_info[i][0]+'_'+connection_info[i][2],1)#输入path 获取dbcreate脚本
    len_script=len(script)
    for j in range(0,len_script):
        print(script[j],file=text)#将dbcreate脚本字符串写进text中输出
text.close()

[8, 3]
[16, 8]
[0, 4]
[8, 9]
[[8, 9], [8, 8], [9, 8], [10, 8], [11, 8], [12, 8], [13, 8], [14, 8], [15, 8], [16, 8]]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  1  1  2  0  0  0  0  0  0  0  0  1  1  0  0 +
+ 0  0  0  0  0  0  0  0  2  2  2  2  2  2  2  2  2  1  1  0  0 +
+ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 +
+ 0  0  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 +
+ 0  0  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 +
+ 0  0  1  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 +
+ 0  0  0  0  0  0  0  0 